In [1]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from underwriting import inventory_freshness_analysis_query as ifa

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [7]:
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'
today = date.today()

In [4]:
query_join = ifa.create_inventory_days_since_package_metric_query(
    ['EL', '99HT'], ['C9-0000370-LIC', 'C10-0000279-LIC'], True, SALES_TRANSACTIONS_START_DATE)
pd.read_sql_query(query_join, engine).head()
#print(query_join)

,license_number,receipt_number,rt_type,sales_customer_type,sales_datetime,sales_month,total_packages,rt_total_price,tx_type,tx_package_id,tx_package_label,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price,price_per_unit,package_id,package_label,type,packaged_date,last_modified_at,package_type,product_name,product_category_name,quantity,unit_of_measure,item_id,item_product_category_type,production_batch_number,source_production_batch_numbers,source_harvest_names,is_testing_sample,is_trade_sample,is_on_hold,archived_date,finished_date,sales_days_diff,inventory_days_diff
0,C10-0000279-LIC,0182037571,active,Consumer,2022-04-21 19:41:26+00:00,2022-04-01 00:00:00+00:00,2,125.0,active,24368112,1A4060300003EE8000133701,"Releaf Balm, 15ml Finished Good, 3CBD:1THC",Topical (volume - each),Each,1.0,25.0,25.0,24368112,1A4060300003EE8000133701,active,2022-04-17,2022-04-22 11:19:50+00:00,Product,"Releaf Balm, 15ml Finished Good, 3CBD:1THC",Topical (volume - each),33.0,Each,2002601,InfusedNonEdible,,012822PB001,"2020-11-02-dry room 1-H, 2020-11-03-dry room 1...",False,False,False,None,None,4,5
1,C10-0000279-LIC,0182037497,active,Consumer,2022-04-21 16:11:53+00:00,2022-04-01 00:00:00+00:00,2,115.0,active,24368121,1A4060300003EE8000133710,"Releaf Balm, 50ml Finished Good, 1CBD:3THC",Topical (volume - each),Each,1.0,75.0,75.0,24368121,1A4060300003EE8000133710,active,2022-04-17,2022-04-22 11:19:49+00:00,Product,"Releaf Balm, 50ml Finished Good, 1CBD:3THC",Topical (volume - each),34.0,Each,2002564,InfusedNonEdible,,021822PB001,"2020-11-02-dry room 1-H, 2020-11-03-dry room 1...",False,False,False,None,None,4,5
2,C10-0000279-LIC,0182033263,active,Consumer,2022-04-21 11:15:07+00:00,2022-04-01 00:00:00+00:00,3,235.0,active,24368121,1A4060300003EE8000133710,"Releaf Balm, 50ml Finished Good, 1CBD:3THC",Topical (volume - each),Each,1.0,75.0,75.0,24368121,1A4060300003EE8000133710,active,2022-04-17,2022-04-22 11:19:49+00:00,Product,"Releaf Balm, 50ml Finished Good, 1CBD:3THC",Topical (volume - each),34.0,Each,2002564,InfusedNonEdible,,021822PB001,"2020-11-02-dry room 1-H, 2020-11-03-dry room 1...",False,False,False,None,None,4,5
3,C10-0000279-LIC,0182033263,active,Consumer,2022-04-21 11:15:07+00:00,2022-04-01 00:00:00+00:00,3,235.0,active,24368116,1A4060300003EE8000133705,"Releaf Capsule, 30ct Finished Good, 2CBD:4THC:...",Capsule (weight - each),Each,1.0,85.0,85.0,24368116,1A4060300003EE8000133705,active,2022-04-17,2022-04-22 11:19:52+00:00,Product,"Releaf Capsule, 30ct Finished Good, 2CBD:4THC:...",Capsule (weight - each),21.0,Each,3243021,Concentrate,,020222CP003,"2019-09-06-Processing-H, 2019-09-08-Processing...",False,False,False,None,None,4,5
4,C10-0000279-LIC,0182037624,active,Consumer,2022-04-21 18:13:20+00:00,2022-04-01 00:00:00+00:00,5,365.0,active,24368116,1A4060300003EE8000133705,"Releaf Capsule, 30ct Finished Good, 2CBD:4THC:...",Capsule (weight - each),Each,1.0,85.0,85.0,24368116,1A4060300003EE8000133705,active,2022-04-17,2022-04-22 11:19:52+00:00,Product,"Releaf Capsule, 30ct Finished Good, 2CBD:4THC:...",Capsule (weight - each),21.0,Each,3243021,Concentrate,,020222CP003,"2019-09-06-Processing-H, 2019-09-08-Processing...",False,False,False,None,None,4,5


In [5]:
query_metric = ifa.create_company_freshness_metric_query(['EL'], ['C9-0000370-LIC'], True, SALES_TRANSACTIONS_START_DATE, 'tx_product_category_name')
print(query_metric)


        SELECT 
            tx_product_category_name,
            AVG(CASE WHEN sales_days_diff > 0 THEN sales_days_diff ELSE 0 END) AS avg_days_since_sale,
            MAX(CASE WHEN sales_days_diff > 0 THEN sales_days_diff ELSE 0 END) AS max_days_since_sale,
            MIN(CASE WHEN sales_days_diff > 0 THEN sales_days_diff ELSE 0 END) AS min_days_since_sale,
            COUNT(tx_product_category_name) AS number_of_sales,
            AVG(price_per_unit) AS avg_sale_price
        FROM 
            (
        SELECT 
            sales.license_number,
            sales.receipt_number,
            sales.rt_type,
            sales.sales_customer_type,
            sales.sales_datetime,
            sales.sales_month,
            sales.total_packages,
            sales.rt_total_price,
            sales.tx_type,
            sales.tx_package_id,
            sales.tx_package_label,
            sales.tx_product_name,
            sales.tx_product_category_name,
            sales.tx_unit_of_measure

In [6]:
pd.read_sql_query(query_metric, engine)

,tx_product_category_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,avg_sale_price
0,Tincture (volume - each),404.461538,512,76,13,46.006154
1,Extract (weight - each),105.798817,386,2,169,30.546036
2,Flower (packaged gram - each),43.121212,86,8,33,1.000000
3,Capsule (weight - each),116.379310,281,60,29,3.427586
4,Flower (packaged - each),73.593220,306,2,59,38.508475
5,Topical (volume - each),101.437500,185,11,16,66.281250
6,Edible (weight - each),99.640596,548,2,537,17.260521
7,Flower (packaged ounce - each),43.777778,137,12,9,185.555556
8,Vape Cartridge (weight - each),72.133929,183,1,112,32.035357
9,Edible (volume - each),3.000000,4,2,2,1.000000


In [7]:
query_metric = ifa.create_company_freshness_metric_query(
    ['EL', '99HT'],
    ['C9-0000370-LIC', 'C10-0000279-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, 'tx_product_category_name')
pd.read_sql_query(query_metric, engine)

,tx_product_category_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,avg_sale_price
0,Edible (weight - each),52.032041,548,0,4307,21.994617
1,Flower (packaged quarter - each),38.043956,120,4,182,84.244505
2,Vape Cartridge (weight - each),65.442829,276,0,1032,47.718178
3,Edible (volume - each),70.138425,268,0,419,14.964224
4,Extract (weight - each),88.232877,386,1,511,36.169824
5,Capsule (weight - each),94.565217,281,0,69,44.266667
6,Vape Cartridge (volume - each),34.541387,323,1,894,51.100861
7,Pre-Roll Flower,51.134615,415,0,3068,19.977341
8,Seeds (each),96.000000,154,33,4,121.875000
9,Tincture (volume - each),113.728625,512,1,538,61.435325


In [1]:
query_metric = ifa.create_company_freshness_metric_query(
    ['EL', '99HT'],
    ['C9-0000370-LIC', 'C10-0000279-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, 'tx_product_category_name')
pd.read_sql_query(query_metric, engine)

NameError: name 'ifa' is not defined

In [9]:
query_valuation = ifa.create_company_inventory_valudation_by_last_sale_price(
    ['EL'],
    ['C9-0000370-LIC'],
    True, SALES_TRANSACTIONS_START_DATE)
pd.read_sql_query(query_valuation, engine)

,total_inventory_valuation
0,57203.76


In [10]:
query_valuation = ifa.create_company_inventory_valudation_by_last_sale_price(
    ['EL'],
    ['C9-0000370-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, False)
pd.read_sql_query(query_valuation, engine)

,total_inventory_valuation
0,50897.76


In [11]:
query_top_product_by_speed = ifa.best_selling_products_by_liquidity(
    ['EL', '99HT'],
    ['C9-0000370-LIC', 'C10-0000279-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, 'tx_product_name',
    'avg_days_since_sale', 'ASC', '10')
pd.read_sql_query(query_top_product_by_speed, engine)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,avg_sale_price
0,State Flower - Jarred Flower - 3.5 gram - Gela...,4.964286,9,1,28,70.000000
1,19023 OASIS Chocolate Peppermint Patties 100mg...,5.000000,8,1,19,24.802632
2,MVN - Blueberry Skittles - Eighth -PP,5.000000,9,1,24,69.416667
3,Source Flower Wookies Quarter 7g,6.000000,11,4,12,94.208333
4,"Eureka Kira Kira Diamond Infused Pre-Roll, Sun...",6.608696,10,3,23,55.000000
5,SF-Lambsbread-S-F-14.0,6.909091,14,1,11,173.181818
6,"Nightcap Kira Kira Diamond Infused Pre-Roll, S...",6.937500,10,3,16,55.000000
7,SF-GDP-I-F-14.0,8.000000,15,3,11,175.000000
8,Indoor - Connected - Eighth - Pantera Limone,8.538462,17,3,13,74.423077
9,Indoor - Connected - Pre-Roll Single - Pantera...,9.125000,18,3,16,18.000000


In [12]:
query_top_product_by_price = ifa.best_selling_products_by_liquidity(
    ['EL', '99HT'],
    ['C9-0000370-LIC', 'C10-0000279-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, 'tx_product_name',
    'avg_sale_price', 'DESC', '10')
pd.read_sql_query(query_top_product_by_price, engine)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,avg_sale_price
0,budbud Cosmic Glue 14g,37.595238,64,7,42,193.607143
1,Claybourne Co. Premium Small Bud 14g - (Case o...,45.472222,70,12,36,184.763889
2,Day Dreams Pouch (Sativa) Crescendo 14g,30.522727,66,6,44,183.738636
3,SF-GDP-I-F-14.0,8.000000,15,3,11,175.000000
4,SF-Lambsbread-S-F-14.0,6.909091,14,1,11,173.181818
5,Claybourne Co. Premium Small Bud 14g - (Case o...,19.555556,36,2,27,164.814815
6,budbud Dragonfruit 14g,62.109091,125,2,55,162.754545
7,"MR. ZIPS | Sunshine Punch | 14g Flower, Fort P...",20.291667,40,5,24,155.000000
8,MM-FloWhite-F-100T-14.18,74.466667,133,3,30,147.483333
9,Orange Sunset Smalls Bags 14 g Case 224 g,96.384615,127,58,13,128.076923


In [9]:
query_valuation_time_discount = ifa.create_company_inventory_valudation_by_discounting_time_value(
    ['EL'],
    ['C9-0000370-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, .2)
pd.read_sql_query(query_valuation_time_discount, engine)

,discounted_price,package_id,product_name,quantity,inventory_year_diff,avg_price_per_unit
0,47.208749,20887745,FloraCal Kush Mints Prepackaged 1/8th*,28.0,0.315068,50.000000
1,29.407330,8164568,STIIIZY - Purple Punch Pod - 0.5g,0.0,0.378082,31.505952
2,23.232177,12801684,Kiva Bar Dark Chocolate CBD - 100mg,1.0,1.060274,28.186667
3,44.330687,23614657,Surplus Purified Live Resin: 1g Cart: Kush Min...,21.0,0.082192,45.000000
4,30.348268,12801666,Level Protab Hangover,34.0,0.846575,35.413333
...,...,...,...,...,...,...
382,19.980030,22290643,KE - THC Indica Mango 100mg,5.0,0.005479,20.000000
383,19.332073,22292607,KE - THC NANO - Cran Pomegranate,11.0,0.186301,20.000000
384,NaN,8164578,Honeyleaf Cartridge - Super Silver Haze - 0.5g,0.0,0.290411,NaN
385,28.089554,20887717,BeMore Blunt Lemon Bean x Muffinz Sauce (1.5g)...,1.0,0.345205,29.914286


In [5]:
# 100 qt, 10 d, 1000

# 365 days

n = 1

1000 / (1+.2) ** n

833.3333333333334

In [14]:
query_valuation_time_discount = ifa.create_company_inventory_valudation_by_discounting_time_value(
    ['EL'],
    ['C9-0000370-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, .2, False)
pd.read_sql_query(query_valuation_time_discount, engine)

,total_inventory_valuation
0,55769.314802


In [15]:
query_join = ifa.create_inventory_days_since_package_metric_query_by_product_name(
    ['EL'], ['C9-0000370-LIC'], True, SALES_TRANSACTIONS_START_DATE)
pd.read_sql_query(query_join, engine)

,license_number,receipt_number,rt_type,sales_customer_type,sales_datetime,sales_month,total_packages,rt_total_price,tx_type,tx_package_id,tx_package_label,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price,price_per_unit,package_id,package_label,type,packaged_date,last_modified_at,package_type,product_name,product_category_name,quantity,unit_of_measure,item_id,item_product_category_type,production_batch_number,source_production_batch_numbers,source_harvest_names,is_testing_sample,is_trade_sample,is_on_hold,archived_date,finished_date,sales_days_diff,inventory_days_diff
0,C9-0000370-LIC,0178610466,active,Consumer,2022-04-08 17:18:04+00:00,2022-04-01 00:00:00+00:00,3.0,81.0,active,23614657,1A406030003BD6D000029677,Surplus Purified Live Resin: 1g Cart: Kush Min...,Vape Cartridge (volume - each),Each,1.0,45.0,45.0,23614657,1A406030003BD6D000029677,active,2022-03-28,2022-04-27 11:18:25+00:00,Product,Surplus Purified Live Resin: 1g Cart: Kush Min...,Vape Cartridge (volume - each),21.0,Each,4094390,Concentrate,,F6000011486PB,"0103SundaeDriver090219A, 0103SundaeDriver09021...",False,False,False,None,None,11.0,30
1,C9-0000370-LIC,0180462839,active,Consumer,2022-04-15 11:42:56+00:00,2022-04-01 00:00:00+00:00,2.0,115.0,active,23614657,1A406030003BD6D000029677,Surplus Purified Live Resin: 1g Cart: Kush Min...,Vape Cartridge (volume - each),Each,1.0,45.0,45.0,23614657,1A406030003BD6D000029677,active,2022-03-28,2022-04-27 11:18:25+00:00,Product,Surplus Purified Live Resin: 1g Cart: Kush Min...,Vape Cartridge (volume - each),21.0,Each,4094390,Concentrate,,F6000011486PB,"0103SundaeDriver090219A, 0103SundaeDriver09021...",False,False,False,None,None,18.0,30
2,C9-0000370-LIC,0182034643,active,Consumer,2022-04-21 15:34:20+00:00,2022-04-01 00:00:00+00:00,2.0,65.0,active,23614657,1A406030003BD6D000029677,Surplus Purified Live Resin: 1g Cart: Kush Min...,Vape Cartridge (volume - each),Each,1.0,45.0,45.0,23614657,1A406030003BD6D000029677,active,2022-03-28,2022-04-27 11:18:25+00:00,Product,Surplus Purified Live Resin: 1g Cart: Kush Min...,Vape Cartridge (volume - each),21.0,Each,4094390,Concentrate,,F6000011486PB,"0103SundaeDriver090219A, 0103SundaeDriver09021...",False,False,False,None,None,24.0,30
3,C9-0000370-LIC,0183122172,active,Consumer,2022-04-26 15:10:26+00:00,2022-04-01 00:00:00+00:00,2.0,85.0,active,23614657,1A406030003BD6D000029677,Surplus Purified Live Resin: 1g Cart: Kush Min...,Vape Cartridge (volume - each),Each,1.0,45.0,45.0,23614657,1A406030003BD6D000029677,active,2022-03-28,2022-04-27 11:18:25+00:00,Product,Surplus Purified Live Resin: 1g Cart: Kush Min...,Vape Cartridge (volume - each),21.0,Each,4094390,Concentrate,,F6000011486PB,"0103SundaeDriver090219A, 0103SundaeDriver09021...",False,False,False,None,None,29.0,30
4,C9-0000370-LIC,0176984682,active,Consumer,2022-04-01 14:41:10+00:00,2022-04-01 00:00:00+00:00,3.0,3.0,active,23621960,1A406030003BD6D000030024,Surplus: 100mg Shot: Citrus Storm,Edible (volume - each),Each,1.0,1.0,1.0,23621960,1A406030003BD6D000030024,active,2022-03-28,2022-04-02 11:23:39+00:00,Product,Surplus: 100mg Shot: Citrus Storm,Edible (volume - each),1.0,Each,4383169,InfusedEdible,,SUSCI-01,"1-3, 10-5, 11a, 12a, 12b, 13a, 13b, 2020-03-16...",False,False,False,None,None,4.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4859,C9-0000370-LIC,0122019493,active,Consumer,2021-08-14 18:20:07+00:00,2021-08-01 00:00:00+00:00,5.0,125.0,active,16926663,1A4060300006E2E000097383,KE - THC Sativa Ruby Grapefruit 100mg - 43g,Edible (weight - each),Each,2.0,40.0,20.0,7628134,1A4060300006E2E000042536,active,2020-09-01,2020-09-12 22:58:06+00:00,Product,KE - THC Sativa Ruby Grapefruit 100mg - 43g,Edible (weight - each),0.0,Each,383497,InfusedEdible,,,"1st, 2019-10-18-2-H, 2019-10-18-3-H, 2019-10-1...",False,False,False,None,None,347.0,11
4860,C9-0000370-LIC,0099747203,active,Co

In [16]:
query_join = ifa.create_inventory_days_since_package_metric_query(
    ['EL'], ['C9-0000370-LIC'], True, SALES_TRANSACTIONS_START_DATE)
pd.read_sql_query(query_join, engine)

,license_number,receipt_number,rt_type,sales_customer_type,sales_datetime,sales_month,total_packages,rt_total_price,tx_type,tx_package_id,tx_package_label,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price,price_per_unit,package_id,package_label,type,packaged_date,last_modified_at,package_type,product_name,product_category_name,quantity,unit_of_measure,item_id,item_product_category_type,production_batch_number,source_production_batch_numbers,source_harvest_names,is_testing_sample,is_trade_sample,is_on_hold,archived_date,finished_date,sales_days_diff,inventory_days_diff
0,C9-0000370-LIC,0178894526,active,Consumer,2022-04-09 20:39:04+00:00,2022-04-01 00:00:00+00:00,1,40.0,active,23614643,1A406030003BD6D000029668,Surplus Tropics: 1g Cart: Strawberry Daze [S],Vape Cartridge (volume - each),Each,1.0,40.0,40.0,23614643,1A406030003BD6D000029668,active,2022-03-28,2022-04-29 11:16:51+00:00,Product,Surplus Tropics: 1g Cart: Strawberry Daze [S],Vape Cartridge (volume - each),1.0,Each,4094913,Concentrate,,F6000014589PB,"BD-10.20.20-BN5492, BD-10.20.20-BN5507, BD-10....",False,False,False,None,None,12,32
1,C9-0000370-LIC,0182921895,active,Consumer,2022-04-25 19:06:27+00:00,2022-04-01 00:00:00+00:00,1,40.0,active,23614632,1A406030003BD6D000029666,Surplus: 1g Cart: Trainwreck [H],Vape Cartridge (volume - each),Each,1.0,40.0,40.0,23614632,1A406030003BD6D000029666,active,2022-03-28,2022-04-28 11:18:09+00:00,Product,Surplus: 1g Cart: Trainwreck [H],Vape Cartridge (volume - each),9.0,Each,4094922,Concentrate,,F6000015329PB,"5-14-2021 Purple Punch Zone 11, 5-27-21 Purple...",False,False,False,None,None,28,31
2,C9-0000370-LIC,0180714442,active,Consumer,2022-04-16 13:07:37+00:00,2022-04-01 00:00:00+00:00,6,190.0,active,23614630,1A406030003BD6D000029665,Surplus: 1g Cart: Purple Lemonade [S],Vape Cartridge (volume - each),Each,1.0,40.0,40.0,23614630,1A406030003BD6D000029665,active,2022-03-28,2022-04-26 11:18:58+00:00,Product,Surplus: 1g Cart: Purple Lemonade [S],Vape Cartridge (volume - each),3.0,Each,4094920,Concentrate,,F6000014620PB,"201110-024-3209-PAP-01, 2020-02-21-Dry Room 1-...",False,False,False,None,None,19,29
3,C9-0000370-LIC,0180714442,active,Consumer,2022-04-16 13:07:37+00:00,2022-04-01 00:00:00+00:00,6,190.0,active,23614643,1A406030003BD6D000029668,Surplus Tropics: 1g Cart: Strawberry Daze [S],Vape Cartridge (volume - each),Each,1.0,40.0,40.0,23614643,1A406030003BD6D000029668,active,2022-03-28,2022-04-29 11:16:51+00:00,Product,Surplus Tropics: 1g Cart: Strawberry Daze [S],Vape Cartridge (volume - each),1.0,Each,4094913,Concentrate,,F6000014589PB,"BD-10.20.20-BN5492, BD-10.20.20-BN5507, BD-10....",False,False,False,None,None,19,32
4,C9-0000370-LIC,0180916582,active,Consumer,2022-04-17 11:42:19+00:00,2022-04-01 00:00:00+00:00,5,195.0,active,23614630,1A406030003BD6D000029665,Surplus: 1g Cart: Purple Lemonade [S],Vape Cartridge (volume - each),Each,1.0,40.0,40.0,23614630,1A406030003BD6D000029665,active,2022-03-28,2022-04-26 11:18:58+00:00,Product,Surplus: 1g Cart: Purple Lemonade [S],Vape Cartridge (volume - each),3.0,Each,4094920,Concentrate,,F6000014620PB,"201110-024-3209-PAP-01, 2020-02-21-Dry Room 1-...",False,False,False,None,None,20,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1862,C9-0000370-LIC,0104188250,active,Consumer,2021-05-30 19:48:52+00:00,2021-05-01 00:00:00+00:00,3,46.0,active,7903524,1A406030000A2C9000007835,Kushy Punch Sativa Gummy,Edible (weight - each),Each,1.0,18.0,18.0,7903524,1A406030000A2C9000007835,active,2020-09-14,2022-01-30 12:19:20+00:00,Product,Kushy Punch Sativa Gummy,Edible (weight - each),5.0,Each,1329552,InfusedEdible,,,"2019-08-10-Barn Doobland-H, 2019-09-20-Barn Do...",False,False,False,None,None,258,503
1863,C9-0000370-LIC,0104403787,active,Consumer,2021-05-31 14:22:28+00:00,2021-05-01 00:00:00+00:00,2,64.0,active,7903524,1A406030000A2C9000007835,